In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
# new files detection in raw folder
files_info = dbutils.fs.ls("gs://nyc_taxi_analysis_project/raw/")
file_paths = [f.path for f in files_info if f.path.endswith(".parquet")]
processed_files = spark.table("nyc_taxi.bronze.metadata_processed_files")\
    .select("file_name").collect()
processed_file_names = [row.file_name for row in processed_files]
new_files = [f for f in file_paths if f not in processed_file_names]

In [0]:
bronze=spark.read.format('parquet').load(new_files)

In [0]:
# adding metadata
bronze = bronze.withColumn("pickup_year", year("lpep_pickup_datetime")) \
    .withColumn("pickup_month", month("lpep_pickup_datetime")) \
    .withColumn("source_file", col("_metadata.file_path")) \
    .withColumn("ingestion_time", current_timestamp())

In [0]:
bronze.write.mode('append').format('delta').partitionBy('pickup_year','pickup_month').saveAsTable('nyc_taxi.bronze.data')

In [0]:
df_processed=bronze.select('source_file').distinct()
df_processed=df_processed.withColumnRenamed('source_file','file_name')
df_processed.write.mode('append').format('delta').saveAsTable('nyc_taxi.bronze.metadata_processed_files')
